
# <p style="text-align:center"> <font color='darkorange'>**CUNEF**</font>
## <p style="text-align:center"> **TFM - Análisis de sentimiento pólitico en Twitter**
### <p style="text-align:center"> **0. Selección de datos y preparación de variables**</strong><br />
    
<p style="text-align:left">Pablo Mazariegos Reviriego - <font color='orange'>pablo.mazariegos@cunef.edu </font>
    

En este proyecto de Trabajo Fin de Máster, realizaremos un análisis de sentimiento de los tweets hechos por los 5 candidatos políticos a la presidencia de Madrid durante el período de campaña política que abarcó desde el 12 hasta el 27 de mayo de 2023. Utilizaremos una base de datos recopilada manualmente que contiene los tweets de los candidatos. El objetivo principal de este proyecto es desarrollar modelos de aprendizaje automático que puedan clasificar los tweets según su sentimiento (positivo, negativo o neutral).

El proyecto se organizará en diferentes cuadernos, cada uno enfocado en una etapa específica del proceso:

 0. <font color='darkgreen'> **Selección de datos y preparación de variables**</font>
 1. EDA
 2. Word Cloud y Análisis de viralidad
 3. Best Model and Explainability

Este cuaderno se enfoca en el análisis de datos de los tweets de los candidatos políticos durante la campaña electoral de Madrid. Realizaremos un análisis exploratorio de los datos, utilizando técnicas de visualización y evaluando el sentimiento político expresado en los tweets. Además, emplearemos un modelo de sentimiento político previamente entrenado para clasificar los tweets en categorías de sentimiento. Compartiremos nuestros resultados a través de visualizaciones informativas y resumidas, lo que nos permitirá obtener una comprensión más profunda de las opiniones y actitudes de los usuarios durante la campaña política.

  **INDICE:**
 
 - [Importación de Librerias](#0) 
 - [Funciones utilizadas en este notebook](#1) 
 - [Carga de datos](#2)
 - [Exploración de los datos](#3)
 - [Modelo de Sentimiento Pólitico](#4)

  **Correlación:**
 - [Correlación de las variables](#5)
 - [Spearman](#5.1)
 - [Cramer's V](#5.2)
 - [Pearson](#5.3)
 
 
 - [Exportación de los datos](#9)


##  <a name="0"> Importación de Librerias</a>

In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

import warnings
warnings.filterwarnings('ignore')

##  <a name="1">Funciones utilizadas en este notebook</a>

In [2]:
# Definimos una función para preprocesar cada tweet
def preprocess_tweet(tweet):
    # Solo procesamos si el input es de tipo string
    if isinstance(tweet, str):
        tweet = tweet.lower()  # Convertimos el tweet a minúsculas
      
        tweet_words = []
        # Recorremos cada palabra en el tweet
        for word in tweet.split(' '):
            # Si la palabra es una mención a un usuario, la reemplazamos por '@user'
            if word.startswith('@') and len(word) > 1:
                word = '@user'
            # Si la palabra es un enlace, la reemplazamos por 'http'
            elif word.startswith('http'):
                word = "http"
            # Añadimos la palabra a la lista de palabras del tweet
            tweet_words.append(word)

        # Devolvemos el tweet procesado
        return " ".join(tweet_words)
    else:
        # Si el input no es una string, devolvemos una string vacía
        return ""

In [3]:
# Definimos una función para analizar el sentimiento de un tweet
def analyze_sentiment(tweet):
    # Codificamos el tweet para que pueda ser procesado por el modelo
    encoded_tweet = tokenizer(tweet, return_tensors='pt')
    # Obtenemos las puntuaciones de sentimiento del modelo
    output = model(**encoded_tweet)
    # Convertimos las puntuaciones en un array de numpy
    scores = output[0][0].detach().numpy()
    # Convertimos las puntuaciones en probabilidades usando la función softmax
    scores = softmax(scores)
    # Devolvemos un diccionario que asocia cada etiqueta con su probabilidad
    return dict(zip(labels, scores))

In [4]:
def get_corr_matrix(dataset=None, metodo='spearman', size_figure=[10,8]):
    if dataset is None:
        print(u'\nHace falta pasar argumentos a la función')
        return 1
    sns.set(style="white")
    corr = dataset.corr(method=metodo)
    for i in range(corr.shape[0]):
        corr.iloc[i, i] = 0
    f, ax = plt.subplots(figsize=size_figure)
    sns.heatmap(corr, annot=True, fmt=".2f", square=True, linewidths=.5, cmap='coolwarm', vmin=-1, vmax=1)
    plt.show()


In [5]:
def cramers_v(var1,var2):
    """ 
    calculate Cramers V statistic for categorial-categorial association.
    uses correction from Bergsma and Wicher,
    Journal of the Korean Statistical Society 42 (2013): 323-328
    
    confusion_matrix: tabla creada con pd.crosstab()
    
    """
    crosstab =np.array(pd.crosstab(var1,var2, rownames=None, colnames=None))
    chi2 = ss.chi2_contingency(crosstab)[0]
    n = crosstab.sum()
    phi2 = chi2 / n
    r, k = crosstab.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1),(rcorr-1)))

##  <a name="2"> Carga de datos</a>

In [6]:
df = pd.read_excel('../data/raw/tweets_12-27Mayo.xlsx', usecols=lambda x: x != 'Nº')
df.head()

,PARTIDO,CANDIDATO,NICK,FOLLOWERS,FECHA,POST,VIDEO,FOTO,REPOST,RETWEET,WHO,COMMENTS,SHARED,LIKES,VIEWED,VOTOS,PORCENTAJE,ESCAÑOS
0,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Comenzamos la campaña, una vez más, junto a la...",SI,NO,NO,NO,NaN,198,261,1260,58700,1586985,0.4734,71
1,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Madrid es la región del Espíritu de Ermua, la ...",NO,NO,NO,NO,NaN,550,561,2127,101600,1586985,0.4734,70
2,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Majadahonda con ganas de Libertad, familia, un...",NO,SI,NO,NO,NaN,140,213,1042,59700,1586985,0.4734,70
3,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,❤️❤️,NO,SI,SI,NO,@cayetanaAT\n,155,343,2958,159100,1586985,0.4734,70
4,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,Presidente: líbranos del mal.,NO,SI,NO,NO,NaN,893,549,2592,330800,1586985,0.4734,70


##  <a name="3"> Exploración de los datos</a>

El método info() proporciona información esencial sobre tu DataFrame. Este resumen incluye:

1. El número total de filas y la gama de índices.
2. El número total de columnas.
3. El nombre de cada columna, la cantidad de valores no nulos que tiene y su tipo de datos.
4. La cantidad de memoria que utiliza el DataFrame.

In [7]:
print('Count of rows in the data is:  ', len(df))
print('Count of columns in the data is:  ', len(df.columns))
df.info()

Count of rows in the data is:   773
Count of columns in the data is:   18
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   PARTIDO     773 non-null    object        
 1   CANDIDATO   773 non-null    object        
 2   NICK        773 non-null    object        
 3   FOLLOWERS   773 non-null    int64         
 4   FECHA       773 non-null    datetime64[ns]
 5   POST        771 non-null    object        
 6   VIDEO       773 non-null    object        
 7   FOTO        773 non-null    object        
 8   REPOST      773 non-null    object        
 9   RETWEET     773 non-null    object        
 10  WHO         431 non-null    object        
 11  COMMENTS    773 non-null    int64         
 12  SHARED      773 non-null    int64         
 13  LIKES       773 non-null    int64         
 14  VIEWED      773 non-null    int64         
 15  

Comprobamos el total de nulos que hay en cada columna

In [8]:
df.isnull().sum()

PARTIDO         0
CANDIDATO       0
NICK            0
FOLLOWERS       0
FECHA           0
POST            2
VIDEO           0
FOTO            0
REPOST          0
RETWEET         0
WHO           342
COMMENTS        0
SHARED          0
LIKES           0
VIEWED          0
VOTOS           0
PORCENTAJE      0
ESCAÑOS         0
dtype: int64

Se modifican los "NO" por False y "SI" por True de las columnas VIDEO, FOTO, REPOTS, RETWEET y WHO

In [9]:
df.replace({'VIDEO': {'NO': False, 'SI': True},
            'FOTO': {'NO': False, 'SI': True},
            'REPOST': {'NO': False, 'SI': True},
            'RETWEET': {'NO': False, 'SI': True},
            'WHO': {np.nan: False}}, inplace=True)

df['COMMENTS vs VIEWED'] = df['COMMENTS'] / df['VIEWED']
df['SHARED vs VIEWED'] = df['SHARED'] / df['VIEWED']
df['LIKES vs VIEWED'] = df['LIKES'] / df['VIEWED']

In [10]:
df.head()

,PARTIDO,CANDIDATO,NICK,FOLLOWERS,FECHA,POST,VIDEO,FOTO,REPOST,RETWEET,...,COMMENTS,SHARED,LIKES,VIEWED,VOTOS,PORCENTAJE,ESCAÑOS,COMMENTS vs VIEWED,SHARED vs VIEWED,LIKES vs VIEWED
0,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Comenzamos la campaña, una vez más, junto a la...",True,False,False,False,...,198,261,1260,58700,1586985,0.4734,71,0.003373,0.004446,0.021465
1,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Madrid es la región del Espíritu de Ermua, la ...",False,False,False,False,...,550,561,2127,101600,1586985,0.4734,70,0.005413,0.005522,0.020935
2,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Majadahonda con ganas de Libertad, familia, un...",False,True,False,False,...,140,213,1042,59700,1586985,0.4734,70,0.002345,0.003568,0.017454
3,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,❤️❤️,False,True,True,False,...,155,343,2958,159100,1586985,0.4734,70,0.000974,0.002156,0.018592
4,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,Presidente: líbranos del mal.,False,True,False,False,...,893,549,2592,330800,1586985,0.4734,70,0.002700,0.001660,0.007836


Volvemos a comprobar los datos

In [11]:
print('Count of rows in the data is:  ', len(df))
print('Count of columns in the data is:  ', len(df.columns))
df.info()

Count of rows in the data is:   773
Count of columns in the data is:   21
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 773 entries, 0 to 772
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   PARTIDO             773 non-null    object        
 1   CANDIDATO           773 non-null    object        
 2   NICK                773 non-null    object        
 3   FOLLOWERS           773 non-null    int64         
 4   FECHA               773 non-null    datetime64[ns]
 5   POST                771 non-null    object        
 6   VIDEO               773 non-null    bool          
 7   FOTO                773 non-null    bool          
 8   REPOST              773 non-null    bool          
 9   RETWEET             773 non-null    bool          
 10  WHO                 773 non-null    object        
 11  COMMENTS            773 non-null    int64         
 12  SHARED              773 non-null

In [12]:
df.isnull().sum()

PARTIDO               0
CANDIDATO             0
NICK                  0
FOLLOWERS             0
FECHA                 0
POST                  2
VIDEO                 0
FOTO                  0
REPOST                0
RETWEET               0
WHO                   0
COMMENTS              0
SHARED                0
LIKES                 0
VIEWED                0
VOTOS                 0
PORCENTAJE            0
ESCAÑOS               0
COMMENTS vs VIEWED    0
SHARED vs VIEWED      0
LIKES vs VIEWED       0
dtype: int64

##  <a name="4">Modelo de Sentimiento Pólitico</a>

Para añadir el sentimiento de los Post, utilizo un modelo ya entrenado "cardiffnlp/xlm-twitter-politics-sentiment".

Esta es una "extensión" del modelo multilingüe twitter-xlm-roberta-base-sentiment (modelo, artículo original) con un enfoque en el sentimiento de los tweets de los políticos. El ajuste fino del sentimiento original se realizó en 8 idiomas (Ar, En, Fr, De, Hi, It, Sp, Pt), pero se realizó un entrenamiento adicional utilizando tweets de Miembros del Parlamento del Reino Unido (inglés), España (español) y Grecia (griego). Este modelo cuenta con datos hasta el año 2021.

**Se puede encontrar aqui:** https://huggingface.co/cardiffnlp/xlm-twitter-politics-sentiment

**Modelo base Roberta** 

Este es un modelo multilingüe XLM-roBERTa-base entrenado en aproximadamente 198 millones de tweets y afinado para el análisis de sentimientos. El ajuste fino del sentimiento se realizó en 8 idiomas (Ar, En, Fr, De, Hi, It, Sp, Pt), pero puede utilizarse para más idiomas (ver el artículo para más detalles). Este modelo ha sido entrenado con datos hasta el año 2020.

**Se puede encontrar aqui:** https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment

In [13]:
# Definimos el nombre del modelo pre-entrenado que vamos a utilizar
MODEL = "cardiffnlp/xlm-twitter-politics-sentiment"

# Cargamos el modelo para la clasificación de secuencias 
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Cargamos el tokenizador asociado a este modelo
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [14]:
# Extraemos la columna 'POST' del DataFrame original para trabajar con ella
df_post = df['POST']

# Definimos las etiquetas que usará el modelo para la clasificación
labels = ['Negative', 'Neutral', 'Positive']

In [15]:
# Aplicamos la función de preprocesamiento a cada tweet
df_post['processed_text'] = df_post.apply(preprocess_tweet)

In [16]:
# Aplicamos la función de análisis de sentimiento a cada tweet procesado
df_post['sentiment_analysis'] = df_post['processed_text'].apply(analyze_sentiment)

KeyboardInterrupt: 

In [ ]:
# Aquí estamos tomando la columna 'sentiment_analysis' del DataFrame df_post,
# que contiene diccionarios, y la estamos convirtiendo en un DataFrame llamado
# 'sentimiento_df'. Cada clave en los diccionarios se convierte en una columna en el
# nuevo DataFrame. Esto se logra con el método apply(pd.Series).
sentimiento_df = df_post['sentiment_analysis'].apply(pd.Series)

# Aquí añadimos una nueva columna llamada 'sentimiento' al DataFrame sentimiento_df.
# Para cada fila, el valor de esta nueva columna se determina usando el método
# idxmax(axis=1). Este método devuelve el nombre de la columna que tiene el valor
# más alto en esa fila. En otras palabras, estamos seleccionando el sentimiento
# (Negative, Neutral, Positive) que tiene la probabilidad más alta de acuerdo con
# el análisis de sentimientos realizado anteriormente.
sentimiento_df['sentimiento'] = sentimiento_df.idxmax(axis=1)

# Finalmente, mostramos las primeras filas del DataFrame para verificar que todo se haya
# realizado correctamente.
sentimiento_df.head()

In [ ]:
df = pd.concat([df, sentimiento_df], axis=1)

In [ ]:
#Reorganización de las columnas
column_order = ['PARTIDO', 'CANDIDATO', 'NICK', 'FOLLOWERS', 'FECHA', 'POST', 'sentimiento', 'Negative', 'Neutral', 'Positive', 'VIDEO', 'FOTO', 'REPOST', 'RETWEET', 'VIEWED',
                'COMMENTS', 'COMMENTS vs VIEWED', 'SHARED', 'SHARED vs VIEWED', 'LIKES', 'LIKES vs VIEWED', 'VOTOS',
                'PORCENTAJE', 'ESCAÑOS']

df = df.reindex(columns=column_order)

In [ ]:
# Convertir los nombres de las columnas a minúsculas
df.columns = df.columns.str.lower()

Última visualización del data Frame antes de su exportación

In [ ]:
df.head()

# **Correlación**

 ##  <a name="5"> Correlación de las variables</a> 

In [ ]:
plt.figure(figsize=(10, 8))  # Tamaño personalizado en pulgadas (ancho, alto)
correlation = df.corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.show()

## <a name="5.1">Spearman</a> 

In [ ]:
df_spearman = df[['partido', 'candidato', 'nick', 'followers', 'fecha',
       'post', 'sentimiento', 'negative', 'neutral', 'positive', 'video',
       'foto', 'repost', 'retweet', 'viewed', 'comments', 'comments vs viewed',
       'shared', 'shared vs viewed', 'likes', 'likes vs viewed', 'votos',
       'porcentaje', 'escaños']]
df_spearman.head()

In [ ]:
get_corr_matrix(dataset = df_spearman, size_figure = [10,8])

In [ ]:
spearman = df_spearman.corr(method = 'spearman')
print(spearman)

## <a name="5.2">Cramer's V</a> 

In [ ]:
df_categorical_variables = df[['partido', 'candidato', 'nick', 'sentimiento', 'video',
                                               'foto', 'repost', 'retweet']]

df_categorical_variables.head()

In [ ]:
rows = []
for var1 in df_categorical_variables :
  col = []
  for var2 in df_categorical_variables :
    cramers = cramers_v(df_categorical_variables[var1], df_categorical_variables[var2]) # Cramer's V test
    col.append(round(cramers,2)) # Keeping of the rounded value of the Cramer's V  
  rows.append(col)
  
cramers_results = np.array(rows)
df_vcramer = pd.DataFrame(cramers_results, columns = df_categorical_variables .columns,
                          index = df_categorical_variables .columns)

sns.heatmap(df_vcramer, vmin=0, vmax=1, square=True, annot=True, linewidths=.5, cmap='coolwarm', fmt=".2f")


In [ ]:
df_vcramer

##  <a name="5.3">Pearson</a>

In [ ]:
df_continous_vairables = df[['followers', 'negative', 'neutral', 'positive', 'viewed', 'comments', 'comments vs viewed',
       'shared', 'shared vs viewed', 'likes', 'likes vs viewed', 'votos',
       'porcentaje', 'escaños']]
df_continous_vairables.head()

In [ ]:
get_corr_matrix(dataset = df_continous_vairables,
                metodo = 'pearson', size_figure = [10,8])

##  <a name="9"> Exportación de los datos</a>

Seexportan los datos como "df_sentimiento" y en la carpeta de preprocesado

In [ ]:
df.to_csv('../data/processed/df_sentimiento.csv')